In [1]:
import numpy as np
import os
import glog as log
import copy

from __future__ import division

import open3d as o3d
from open3d import JVisualizer
import pandas as pd

from evaluation.tools.mesh import Mesh
from evaluation.tools.mesh_evaluator import MeshEvaluator

# Rotation matrices:
# East North Up (ENU) frame to Unity's world frame of reference
enu_R_unity = np.array([[1, 0, 0],
                        [0, 0, 1],
                        [0, 1, 0]])
unity_R_enu = np.transpose(enu_R_unity)

# Right Handed frame to Unity's Left Handed frame of reference
righthand_R_lefthand = np.array([[1, 0, 0],
                                 [0, -1, 0],
                                 [0, 0, 1]])
lefthand_R_righthand = np.transpose(righthand_R_lefthand)

In [2]:
# FILL PATHS BELOW
#gt_mesh_path = "/home/tonirv/Downloads/tesse_multiscene_office1_3d_semantic_v5.ply"
#est_mesh_path = "/home/tonirv/Downloads/tesse_semantics_2.ply"

#gt_mesh_path = "/home/tonirv/Code/ROS/flight_goggles_ws/src/voxblox/voxblox_ros/mesh_results/semantic_mesh_tonirv_ld_9118_6487309760727328010.ply"
#est_mesh_path = "/home/tonirv/Code/ROS/flight_goggles_ws/src/voxblox/voxblox_ros/mesh_results/semantic_mesh_tonirv_ld_9118_6487309760727328010.ply"

#gt_mesh_path = "/home/tonirv/Downloads/tesse_multiscene_office1_3d_semantic_v5.ply"
#est_mesh_path = "/home/tonirv/Code/ROS/flight_goggles_ws/src/voxblox/voxblox_ros/mesh_results/tesse_semantics_3.ply"

gt_mesh_path = "/home/tonirv/Downloads/office1_tony.ply"
est_mesh_path = "/home/tonirv/Code/ROS/flight_goggles_ws/src/voxblox/voxblox_ros/mesh_results/tesse_tonirv_ld_19379_8746799625546883009.ply"

In [3]:
print("Loading Ground-truth mesh...")
gt_mesh_original = Mesh(gt_mesh_path)
print("Loading Estimated mesh...")
est_mesh_original = Mesh(est_mesh_path)

Loading Ground-truth mesh...
Testing mesh in open3d ...
geometry::TriangleMesh with 1572372 points and 524124 triangles.
[[14.5       4.       27.5     ]
 [13.5       4.       39.5     ]
 [14.5       4.       39.5     ]
 ...
 [ 6.567781  2.118583 36.00596 ]
 [ 6.568742  2.126512 36.21212 ]
 [ 6.568742  2.126512 36.00596 ]]
[[      0       1       2]
 [      3       4       5]
 [      6       7       8]
 ...
 [1572363 1572364 1572365]
 [1572366 1572367 1572368]
 [1572369 1572370 1572371]]

Loading Estimated mesh...
Testing mesh in open3d ...
geometry::TriangleMesh with 460748 points and 487817 triangles.
[[-3.15502 25.675    1.025  ]
 [-3.175   25.675    1.02952]
 [-3.16946 25.625    1.025  ]
 ...
 [17.925   33.5685   1.575  ]
 [17.975   33.5652   1.575  ]
 [18.025   33.5736   1.575  ]]
[[     0      1      2]
 [     2      1      3]
 [     4      5      6]
 ...
 [374700 374637 460746]
 [460747 460737 374700]
 [374700 460737 374751]]



In [4]:
# Transform Meshes to same frame of reference
gt_mesh = copy.deepcopy(gt_mesh_original)
est_mesh = copy.deepcopy(est_mesh_original)

In [5]:
# Align Pointclouds Manually:
#est_mesh.mesh_o3d.translate([0, -5, 0])
#gt_mesh.transform_left(righthand_R_lefthand)
gt_mesh.transform_left(enu_R_unity)

Transforming mesh according to left matrix:
[[1 0 0]
 [0 0 1]
 [0 1 0]]


In [6]:
vis = o3d.visualization.Visualizer()
vis.create_window()
vis.get_render_option().mesh_show_back_face = True
vis.add_geometry(est_mesh.mesh_o3d)
vis.add_geometry(gt_mesh.mesh_o3d)
vis.add_geometry(o3d.geometry.create_mesh_coordinate_frame(size=4))
vis.run()
vis.destroy_window()

In [7]:
NUMBER_OF_SAMPLES=1000000
gt_pcl = o3d.geometry.sample_points_uniformly(gt_mesh.mesh_o3d, NUMBER_OF_SAMPLES)
# Don't sample estimated mesh, just pick vertices, otw you'll be mixing colors...
# est_pcl = o3d.geometry.sample_points_uniformly(est_mesh.mesh_o3d, NUMBER_OF_SAMPLES)
est_pcl = o3d.io.read_point_cloud(est_mesh_path)

In [8]:
# Calculate normals for nice visualization
# THIS COLORS THE PCL?>>>>>????
#o3d.geometry.estimate_normals(
#        est_pcl,
#        search_param=o3d.geometry.KDTreeSearchParamHybrid(radius=0.1,
#                                                          max_nn=5))
#o3d.geometry.estimate_normals(
#        gt_pcl,
#        search_param=o3d.geometry.KDTreeSearchParamHybrid(radius=0.1,
#                                                          max_nn=5))

In [9]:
vis = o3d.visualization.Visualizer()
vis.create_window()
vis.get_render_option().mesh_show_back_face = True
vis.add_geometry(gt_pcl)
vis.add_geometry(est_pcl)
vis.run()
vis.destroy_window()

In [10]:
# ICP
def draw_registration_result(source, target, transformation):
    source_temp = copy.deepcopy(source)
    target_temp = copy.deepcopy(target)
    source_temp.paint_uniform_color([1, 0.706, 0])
    target_temp.paint_uniform_color([0, 0.651, 0.929])
    source_temp.transform(transformation)
    o3d.visualization.draw_geometries([source_temp, target_temp])
def draw_correspondences(source, target, correspondences):
    source_temp = copy.deepcopy(source)
    target_temp = copy.deepcopy(target)
    #source_temp.paint_uniform_color([1, 0.706, 0])
    #target_temp.paint_uniform_color([0, 0.651, 0.929])
    o3d.visualization.draw_geometries([source_temp, #target_temp, 
                                       correspondences])
    

In [11]:
# ICP params
ICP_THRESHOLD = 1.5
trans_init = np.asarray([[1.0, 0.0, 0.0, 0.0],
                         [0.0, 1.0, 0.0, 0.0],
                         [0.0, 0.0, 1.0, 0.0],
                         [0.0, 0.0, 0.0, 1.0]])

In [12]:
# Visualize initial registration problem
draw_registration_result(est_pcl, gt_pcl, trans_init)

In [13]:
# Evaluate current fit between pointclouds
evaluation = o3d.registration.evaluate_registration(est_pcl, gt_pcl, ICP_THRESHOLD, trans_init)

In [14]:
print("Initial registration")
print(evaluation)

Initial registration
registration::RegistrationResult with fitness = 1.000000, inlier_rmse = 0.086388, and correspondence_set size of 460748
Access transformation to get result.


In [15]:
print("Apply point-to-point ICP")
reg_p2p = o3d.registration.registration_icp(
    est_pcl, gt_pcl, ICP_THRESHOLD, trans_init,
    o3d.registration.TransformationEstimationPointToPoint(),
    o3d.registration.ICPConvergenceCriteria(max_iteration = 2000))
correspondences = reg_p2p.correspondence_set

Apply point-to-point ICP


In [16]:
print(reg_p2p)
print("")

print("Transformation is:")
print(reg_p2p.transformation)
print("")

print("Correspondence Set:")
print(reg_p2p.correspondence_set)
print("")

registration::RegistrationResult with fitness = 1.000000, inlier_rmse = 0.084382, and correspondence_set size of 460748
Access transformation to get result.

Transformation is:
[[ 9.99996411e-01  2.62416375e-03 -5.40076771e-04 -5.74876904e-02]
 [-2.62363973e-03  9.99996089e-01  9.68716680e-04  2.93082676e-02]
 [ 5.42616730e-04 -9.67296236e-04  9.99999385e-01  2.33823400e-02]
 [ 0.00000000e+00  0.00000000e+00  0.00000000e+00  1.00000000e+00]]

Correspondence Set:
std::vector<Eigen::Vector2i> with 460748 elements.
Use numpy.asarray() to access data.



In [17]:
# Draw Registration Result
draw_registration_result(est_pcl, gt_pcl, reg_p2p.transformation)

In [18]:
# Draw Only Correspondences
c2c_lines = o3d.geometry.create_line_set_from_point_cloud_correspondences(est_pcl, gt_pcl, correspondences)
o3d.visualization.draw_geometries([c2c_lines])

In [19]:
# Draw PointClouds and Correspondences
draw_correspondences(est_pcl, gt_pcl, c2c_lines)

In [20]:
def calc_corresp(est_pcl, gt_pcl, correspondences):
    total_negative_matches = 0
    total_positive_matches = 0
    total_correspondences = len(correspondences)
    for correspondence in correspondences:
        if np.allclose(est_pcl.colors[correspondence[0]],
                       gt_pcl.colors[correspondence[1]]):
            total_positive_matches += 1
        else:
            total_negative_matches += 1

    print("Positive color matches: ",total_positive_matches)
    print("Negative color matches: ", total_negative_matches)
    print("Total correspondences: ", total_correspondences)
    assert(total_correspondences == total_negative_matches + total_positive_matches)
    print ("Positive: {}  % ".format(total_positive_matches / total_correspondences * 100))
    print ("Negative: {}  % ".format(total_negative_matches / total_correspondences * 100))

In [21]:
calc_corresp(est_pcl, gt_pcl, correspondences)

('Positive color matches: ', 392685)
('Negative color matches: ', 68063)
('Total correspondences: ', 460748)
Positive: 85.2277166694  % 
Negative: 14.7722833306  % 


In [22]:
print est_pcl.colors[0]

[0.6627451 0.6627451 0.6627451]


In [23]:
# Import Semantic Labels
df = pd.read_csv('/home/tonirv/Code/ROS/flight_goggles_ws/src/voxblox/voxblox_ros/cfg/tesse_multiscene_office1_segmentation_mapping.csv')
df

,name,red,green,blue,alpha,id
0,Airventilation_3x,255,20,127,255,1
1,Airventilation_CrossSection,255,20,127,255,1
2,AirventilationGrid_A,255,20,127,255,1
3,AirventilationGrid_B,255,20,127,255,1
4,Books,250,50,50,255,2
5,Bookshelf,250,50,50,255,2
6,Bottle,255,20,127,255,1
7,Box_Black,255,20,127,255,1
8,Box_Cardboard,255,20,127,255,1
9,Box_Gray,255,20,127,255,1


In [24]:
normalized_df = copy.deepcopy(df)
normalized_df['normalized_red'] = df['red'] / 255
normalized_df['normalized_green'] = df['green'] / 255
normalized_df['normalized_blue'] = df['blue'] / 255

print(sum(np.isclose(normalized_df['normalized_red'], gt_pcl.colors[0][0])))


13


In [25]:
from hashlib import sha1

from numpy import all, array, uint8


class hashable(object):
    r'''Hashable wrapper for ndarray objects.
        Instances of ndarray are not hashable, meaning they cannot be added to
        sets, nor used as keys in dictionaries. This is by design - ndarray
        objects are mutable, and therefore cannot reliably implement the
        __hash__() method.
        The hashable class allows a way around this limitation. It implements
        the required methods for hashable objects in terms of an encapsulated
        ndarray object. This can be either a copied instance (which is safer)
        or the original object (which requires the user to be careful enough
        not to modify it).
    '''
    def __init__(self, wrapped, tight=False):
        r'''Creates a new hashable object encapsulating an ndarray.
            wrapped
                The wrapped ndarray.
            tight
                Optional. If True, a copy of the input ndaray is created.
                Defaults to False.
        '''
        self.__tight = tight
        self.__wrapped = array(wrapped) if tight else wrapped
        self.__hash = int(sha1(wrapped.view(uint8)).hexdigest(), 16)

    def __eq__(self, other):
        return all(self.__wrapped == other.__wrapped)

    def __hash__(self):
        return self.__hash

    def unwrap(self):
        r'''Returns the encapsulated ndarray.
            If the wrapper is "tight", a copy of the encapsulated ndarray is
            returned. Otherwise, the encapsulated ndarray itself is returned.
        '''
        if self.__tight:
            return array(self.__wrapped)



In [26]:
# No need to hash, we have ids now
red_col = df['red'].to_numpy()
blue_col = df['blue'].to_numpy()
green_col = df['green'].to_numpy()

def f(x):    
    return str(x['normalized_red'])+str(x['normalized_green'])+str(x['normalized_blue'])

hashed_df = copy.deepcopy(normalized_df)
hashed_df['hash'] = hashed_df.apply(f, axis=1)
hashed_df

,name,red,green,blue,alpha,id,normalized_red,normalized_green,normalized_blue,hash
0,Airventilation_3x,255,20,127,255,1,1.000000,0.078431,0.498039,1.00.0784313725490.498039215686
1,Airventilation_CrossSection,255,20,127,255,1,1.000000,0.078431,0.498039,1.00.0784313725490.498039215686
2,AirventilationGrid_A,255,20,127,255,1,1.000000,0.078431,0.498039,1.00.0784313725490.498039215686
3,AirventilationGrid_B,255,20,127,255,1,1.000000,0.078431,0.498039,1.00.0784313725490.498039215686
4,Books,250,50,50,255,2,0.980392,0.196078,0.196078,0.9803921568630.1960784313730.196078431373
5,Bookshelf,250,50,50,255,2,0.980392,0.196078,0.196078,0.9803921568630.1960784313730.196078431373
6,Bottle,255,20,127,255,1,1.000000,0.078431,0.498039,1.00.0784313725490.498039215686
7,Box_Black,255,20,127,255,1,1.000000,0.078431,0.498039,1.00.0784313725490.498039215686
8,Box_Cardboard,255,20,127,255,1,1.000000,0.078431,0.498039,1.00.0784313725490.498039215686
9,Box_Gray,255,20,127,255,1,1.000000,0.078431,0.498039,1.00.0784313725490.498039215686


In [27]:
# Generate table from color to id.
def label_from_color(color):
    norm_r = color[0]
    norm_g = color[1]
    norm_b = color[2]
    # TODO(Toni): can be greatly optimized...
    # TODO(Toni): you are comparing floats with == ......
    return normalized_df.loc[(normalized_df['normalized_red'] == norm_r) & 
                  (normalized_df['normalized_green'] == norm_g) & 
                  (normalized_df['normalized_blue'] == norm_b)]['id'].unique().tolist()[0]

In [28]:
# Compare labels between correspondences:
total_label_correspondences = {i:0 for i in normalized_df['id'].unique()}
# Initialize dictionaries to 0:
total_positive_matches = {i:0 for i in normalized_df['id'].unique()}
total_negative_matches = copy.deepcopy(total_positive_matches)

In [29]:
def calc_per_label_corresp():
    for correspondence in correspondences:
        est_pcl_color = est_pcl.colors[correspondence[0]]
        color_hash = str(est_pcl_color[0])+str(est_pcl_color[1])+str(est_pcl_color[2])
        label_id = label_from_color(est_pcl_color)
        if np.allclose(est_pcl_color, gt_pcl.colors[correspondence[1]]):
            total_positive_matches[label_id] += 1
        else:
            total_negative_matches[label_id] += 1

In [ ]:
calc_per_label_corresp()

In [ ]:
print "Dict for total_positive_matches: \n", total_positive_matches
print "Dict for total_negative_matches: \n", total_negative_matches
for label_id in df['id'].unique().tolist():
    total_label_correspondences[label_id] = (total_positive_matches[label_id] + total_negative_matches[label_id])
print "Dict for total_label_correspondences: \n", total_label_correspondences
for label_id in df['id'].unique().tolist():
    print ("ID: {}".format(label_id))
    print ("Positive: {}  % ".format(total_positive_matches[label_id] / total_label_correspondences[label_id] * 100))
    print ("Negative: {}  % ".format(total_negative_matches[label_id] / total_label_correspondences[label_id] * 100))